In [1]:
import requests
from Levenshtein import ratio
import pandas as pd
from datetime import datetime, timedelta
import re
import time
import sys
import os
from joblib import dump, load
from tqdm import tqdm
from pathlib import Path


# Add parent directory to sys.path for config imports
parent_dir = str(Path().resolve().parent)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config.settings import API_KEYS, API_ENDPOINTS

print("📊 Solana DeFi Tracker - Data Collection")
print(f"Cache directory: {os.path.normpath('../data')}")
print(f"Collection timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📊 Solana DeFi Tracker - Data Collection
Cache directory: ..\data
Collection timestamp: 2025-09-02 10:02:40


#### Verify Cache Directories

In [2]:
# Verify cache directories exist, create if missing
cache_dirs = ['../data/api_responses', '../data/processed', '../data/temp']
for cache_dir in cache_dirs:
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir, exist_ok=True)
        print(f"✅ Created cache directory: {cache_dir}")
    else:
        print(f"✅ Cache directory exists: {cache_dir}")

✅ Cache directory exists: ../data/api_responses
✅ Cache directory exists: ../data/processed
✅ Cache directory exists: ../data/temp


#### Helper Functions

In [3]:
def make_request(url, headers=None, params=None, max_retries=3, is_post=False):
    """Make API request with retry logic"""
    for attempt in range(max_retries):
        try:
            if is_post:
                response = requests.post(url, headers=headers, json=params, timeout=30)
            else:
                response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                return None

def save_raw_data(data, filename, description=""):
    """Save raw API response to api_responses directory"""
    filepath = os.path.normpath(f"../data/api_responses/{filename}")
    dump(data, filepath)
    print(f"💾 Saved raw: {description} → {filepath}")
    return filepath

def save_processed_data(data, filename, description=""):
    """Save processed data to processed directory"""
    filepath = os.path.normpath(f"../data/processed/{filename}")
    dump(data, filepath)
    print(f"💾 Saved processed: {description} → {filepath}")
    return filepath

def save_cache(data, filename, description=""):
    """Save cache data to temp directory"""
    filepath = os.path.normpath(f"../data/temp/{filename}")
    dump(data, filepath)
    print(f"💾 Saved cache: {description} → {filepath}")
    return filepath

def load_cache(filename):
    """Load cache data from temp directory"""
    filepath = os.path.normpath(f"../data/temp/{filename}")
    if os.path.exists(filepath):
        try:
            return load(filepath)
        except Exception as e:
            print(f"⚠️ Failed to load cache {filename}: {e}")
            return None
    return None

def format_currency(amount):
    """Format currency amount with appropriate units (K, M, B)"""
    if amount is None or amount == 0:
        return "$0"
    
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.2f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.2f}K"
    else:
        return f"${amount:.2f}"

#### Step 1: Collect DefiLlama Protocol Data (TVL)

In [4]:
print("\n🔍 Collecting Solana Protocol TVL Data...")

def get_all_solana_tvl_data():
    """
    Collect TVL data for Solana DeFi protocols from DefiLlama, excluding CEX and CEX-related protocols.
    
    """
 
    # DefiLlama API endpoint for all protocols
    base_url = API_ENDPOINTS['defillama']['base_url']
    protocols_url = f"{base_url}/protocols"
    
    # Make the API request
    all_protocols = make_request(protocols_url)
    
    if not all_protocols:
        print("❌ DefiLlama API failed")
        return None
    
    print(f"✅ DefiLlama API working! Found {len(all_protocols)} total protocols")
    
    # Comprehensive list of CEX names to exclude
    cex_list = [
        'binance', 'bybit', 'coinbase', 'kraken', 'kucoin', 'okx',
        'crypto.com', 'crypto', 'bitfinex', 'huobi', 'htx', 'gate', 'gate.io',
        'mexc', 'bitget', 'gemini', 'bitstamp', 'bithumb', 'bitpanda',
        'bitmex', 'coinex', 'upbit', 'revolut', 'coindcx', 'bitflyer',
        'coincheck', 'bitbank', 'swissborg', 'deribit'
    ]
    
    # Filter for Solana DeFi protocols (excluding CEX and CEX-related)
    solana_protocols = []
    excluded_protocols = []
    
    for protocol in all_protocols:
        chains = protocol.get('chains', [])
        category = protocol.get('category', '').lower()
        name = protocol.get('name', '').lower()
        
        is_solana = (
            'Solana' in chains or 
            'solana' in chains or
            any('solana' in str(chain).lower() for chain in chains) or
            protocol.get('chain') == 'Solana'
        )
        
        # Exclude CEX and CEX-related protocols
        is_cex_related = (
            category == 'cex' or
            any(cex in name for cex in cex_list)
        )
        
        if is_solana and not is_cex_related:
            tvl_value = protocol.get('tvl') or 0
            
            solana_protocols.append({
                'name': protocol.get('name', 'Unknown'),
                'slug': protocol.get('slug', ''),
                'tvl': tvl_value,
                'chains': chains,
                'category': protocol.get('category', 'Unknown'),
                'change_1h': protocol.get('change_1h'),
                'change_1d': protocol.get('change_1d'),
                'change_7d': protocol.get('change_7d'),
                'mcap': protocol.get('mcap'),
                'symbol': protocol.get('symbol', ''),
                'url': protocol.get('url', ''),
                'description': protocol.get('description', ''),
                'gecko_id': protocol.get("coingeckoId"),
                'timestamp': datetime.now()
            })
        elif is_solana and is_cex_related:
            excluded_protocols.append((protocol.get('name'), category))
    
    if excluded_protocols:
        print(f"⚠️ Excluded {len(excluded_protocols)} CEX-related protocols: {', '.join([name for name, _ in excluded_protocols[:5]])}")
        # Save excluded protocols log to cache directory
        save_cache(excluded_protocols, f'excluded_protocols_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib', 
                    "Excluded CEX protocols log")  
    
    if not solana_protocols:
        print("❌ No Solana DeFi protocols found")
        return None
    
    # Convert to DataFrame
    df = pd.DataFrame(solana_protocols)
    
    # Sort by TVL descending
    df = df.sort_values(by="tvl", ascending=False).reset_index(drop=True)
    
    print(f"🌟 Found {len(df)} Solana DeFi protocols:")
    
    # Calculate statistics
    total_tvl = df['tvl'].sum()
    active_protocols = (df['tvl'] > 0).sum()
    
    print(f"📊 Total Solana DeFi TVL: ${total_tvl:,.0f}")
    print(f"📈 Active protocols (TVL > 0): {active_protocols}/{len(df)}")
    
    # Display top protocols
    print(f"\n{'Rank':<5} {'Protocol':<25} {'TVL':<15} {'Category':<20} {'1d Change':<10}")
    print("=" * 85)
    
    for i, row in df.head(20).iterrows():
        tvl_formatted = format_currency(row['tvl'])
        change_1d = row['change_1d']
        change_str = f"{change_1d:+.1f}%" if change_1d is not None else "N/A"
        
        print(f"{i+1:<5} {row['name'][:24]:<25} {tvl_formatted:<15} "
              f"{row['category'][:19]:<20} {change_str:<10}")
    
    # Show category breakdown
    category_breakdown = (
        df.groupby("category")['tvl']
        .agg(['count', 'sum'])
        .rename(columns={'count': 'protocols', 'sum': 'total_tvl'})
        .sort_values(by="total_tvl", ascending=False)
    )
    
    print(f"\n📋 Category Breakdown:")
    print(f"{'Category':<25} {'Count':<8} {'Total TVL':<15}")
    print("-" * 50)
    
    for category, row in category_breakdown.head(10).iterrows():
        print(f"{category[:24]:<25} {row['protocols']:<8} {format_currency(row['total_tvl']):<15}")
    
    # Save DataFrame to joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'solana_defi_tvl_{timestamp}.joblib'
    save_raw_data(df, filename, 'Solana DeFi protocols TVL data')
    print(f"\n💾 DataFrame saved to {filename}")
    
    return df

# Execute the collection
tvl_df = get_all_solana_tvl_data()

if tvl_df is not None:
    print(f"\n✅ Successfully collected TVL data for {len(tvl_df)} Solana DeFi protocols")
    print("📁 DataFrame saved to joblib file for further analysis")
    print(f"📊 Dataset includes {len(tvl_df)} protocols worth ${tvl_df['tvl'].sum():,.0f} in total TVL")
else:
    print("\n❌ Failed to collect TVL data")

print("\n" + "=" * 50)


🔍 Collecting Solana Protocol TVL Data...
✅ DefiLlama API working! Found 6346 total protocols
⚠️ Excluded 41 CEX-related protocols: Binance CEX, OKX, Bitfinex, Bybit, Gate
💾 Saved cache: Excluded CEX protocols log → ..\data\temp\excluded_protocols_20250902_100252.joblib
🌟 Found 250 Solana DeFi protocols:
📊 Total Solana DeFi TVL: $47,226,721,821
📈 Active protocols (TVL > 0): 219/250

Rank  Protocol                  TVL             Category             1d Change 
1     Lido                      $8.54B          Liquid Staking       -77.9%    
2     Jito Liquid Staking       $3.02B          Liquid Staking       +1.8%     
3     Portal                    $2.72B          Bridge               -1.3%     
4     Kamino Lend               $2.66B          Lending              +2.6%     
5     BlackRock BUIDL           $2.41B          RWA                  +0.0%     
6     Sanctum Validator LSTs    $2.36B          Liquid Staking       +2.5%     
7     Jupiter Perpetual Exchan  $2.31B          Deriva

#### Step 2: Collect DefiLlama Revenue Data

In [5]:
print("\n🔍 Collecting Solana Protocol Revenue Data...")

def get_solana_revenue_data():
    """Collect REVENUE data from DefiLlama"""
    base_url = API_ENDPOINTS['defillama']['base_url']
    revenue_url = f"{base_url}/overview/fees/solana"
    
    #
    params = {
        'dataType': 'dailyRevenue', 
        'excludeTotalDataChart': 'true',
        'excludeTotalDataChartBreakdown': 'true'
    }
    
    data = make_request(revenue_url, params=params)
    
    if not data:
        print("❌ Solana Revenue API failed")
        return None
    
    print(f"✅ Collected Solana revenue data")
    print(f"Total protocols found: {len(data.get('protocols', []))}")
    
    protocols = data.get('protocols', [])
    if not protocols:
        print("No protocol data found.")
        return None
    
    # Sort protocols by total24h in descending order
    sorted_protocols = sorted(protocols, 
                            key=lambda x: x.get('total24h', 0) or 0, 
                            reverse=True)
    
    # Build DataFrame
    revenue_list = []
    for protocol in sorted_protocols:
        revenue_list.append({
            'protocol': protocol.get('name', 'Unknown'),
            'revenue_24h': protocol.get('total24h', 0),  
            'revenue_7d': protocol.get('total7d', 0),    
            'revenue_30d': protocol.get('total30d', 0),  
            'revenue_all_time': protocol.get('totalAllTime', 0),
            'data_type': 'revenue',  
            'chain': 'solana',
            'timestamp': datetime.now()
        })
    
    df = pd.DataFrame(revenue_list)
    
    # Display summary
    protocols_with_data = (df['revenue_24h'] > 0).sum()
    print(f"💵 Found revenue data for {protocols_with_data} active protocols:")
    
    # Show top 10
    top_protocols = df.sort_values(by="revenue_24h", ascending=False).head(10)
    print(f"\n{'Protocol':<25} {'24h Revenue':<15} {'7d Revenue':<15} {'30d Revenue':<15}")
    print("=" * 75)
    
    for _, row in top_protocols.iterrows():
        if row['revenue_24h'] > 0:
            print(f"{row['protocol'][:24]:<25} {format_currency(row['revenue_24h']):<15} "
                  f"{format_currency(row['revenue_7d']):<15} {format_currency(row['revenue_30d']):<15}")
    
    # Save DataFrame to joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'solana_revenue_{timestamp}.joblib'
    save_raw_data(df, filename, 'Solana revenue data')
    print(f"\n💾 DataFrame saved to {filename}")
    
    return df


print("📊 Solana Protocol Revenue Tracker")
print("=" * 50)

revenue_df = get_solana_revenue_data()

if revenue_df is not None:
    print(f"\n✅ Successfully collected revenue data for {len(revenue_df)} protocols")
    print("📁 DataFrame saved to joblib file for further analysis")
else:
    print("\n❌ Failed to collect revenue data")

print("\n" + "=" * 50)


🔍 Collecting Solana Protocol Revenue Data...
📊 Solana Protocol Revenue Tracker
✅ Collected Solana revenue data
Total protocols found: 104
💵 Found revenue data for 92 active protocols:

Protocol                  24h Revenue     7d Revenue      30d Revenue    
Axiom                     $1.38M          $11.23M         $53.25M        
pump.fun                  $1.32M          $11.11M         $42.15M        
Jupiter Perpetual Exchan  $1.14M          $5.91M          $22.74M        
Phantom Wallet            $528.13K        $3.68M          $16.79M        
Solana                    $151.09K        $1.08M          $4.53M         
Meteora DAMM V2           $145.03K        $916.57K        $6.28M         
PumpSwap                  $144.45K        $1.20M          $5.01M         
Raydium AMM               $130.89K        $711.40K        $3.77M         
Photon                    $121.67K        $1.12M          $5.99M         
GMGN                      $103.66K        $739.51K        $3.34M         


#### Step 3: Collect Solana Fees Data

In [7]:
print("\n🔍 Collecting Solana Protocol Fees Data...")

def get_solana_fees_data():
    
    base_url = API_ENDPOINTS['defillama']['base_url']
    fees_url = f"{base_url}/overview/fees/solana"
    
  
    params = {
        'dataType': 'dailyFees',  
        'excludeTotalDataChart': 'true',
        'excludeTotalDataChartBreakdown': 'true'
    }
    
    data = make_request(fees_url, params=params)
    
    if not data:
        print("❌ Solana Fees API failed")
        return None
    
    print(f"✅ Collected Solana fees data")
    print(f"Total protocols found: {len(data.get('protocols', []))}")
    
    protocols = data.get('protocols', [])
    
    if not protocols:
        print("No protocol data found.")
        return None
    
    # Sort protocols by total24h in descending order
    sorted_protocols = sorted(protocols, 
                            key=lambda x: x.get('total24h', 0) or 0, 
                            reverse=True)
    
    # Build list for DataFrame
    fees_list = []
    for protocol in sorted_protocols:
        fees_list.append({
            'protocol': protocol.get('name', 'Unknown'),
            'fees_24h': protocol.get('total24h', 0),     
            'fees_7d': protocol.get('total7d', 0),       
            'fees_30d': protocol.get('total30d', 0),     
            'fees_all_time': protocol.get('totalAllTime', 0),
            'data_type': 'fees',  # Add data type identifier
            'chain': 'solana',
            'timestamp': datetime.now()
        })
    
    # Convert to DataFrame
    df2 = pd.DataFrame(fees_list)
    
    # Display summary
    protocols_with_data = (df2['fees_24h'] > 0).sum()
    print(f"💰 Found fee data for {protocols_with_data} active protocols:")
    
    # Show top 10
    top_protocols = df2.sort_values(by="fees_24h", ascending=False).head(10)
    print(f"\n{'Protocol':<25} {'24h Fees':<15} {'7d Fees':<15} {'30d Fees':<15}")
    print("=" * 70)
    for _, row in top_protocols.iterrows():
        if row['fees_24h'] > 0:
            print(f"{row['protocol'][:24]:<25} {format_currency(row['fees_24h']):<15} "
                  f"{format_currency(row['fees_7d']):<15} {format_currency(row['fees_30d']):<15}")
    
    # Save DataFrame to joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'solana_fees_{timestamp}.joblib'
    save_raw_data(df2, filename, 'Solana fees data')
    print(f"\n💾 DataFrame saved to {filename}")
    
    return df2


print("📊 Solana Protocol Fees Tracker")
print("=" * 50)
    
fees_df = get_solana_fees_data()
    
if fees_df is not None:
    print(f"\n✅ Successfully collected fees data for {len(fees_df)} protocols")
    print("Data saved to joblib file for further analysis")
else:
    print("\n❌ Failed to collect fees data")
    
print("\n" + "=" * 50)


🔍 Collecting Solana Protocol Fees Data...
📊 Solana Protocol Fees Tracker
✅ Collected Solana fees data
Total protocols found: 117
💰 Found fee data for 97 active protocols:

Protocol                  24h Fees        7d Fees         30d Fees       
Jupiter Perpetual Exchan  $4.55M          $23.65M         $90.97M        
Axiom                     $1.38M          $11.23M         $53.25M        
pump.fun                  $1.32M          $11.11M         $42.15M        
Solana                    $1.28M          $9.60M          $41.93M        
Meteora DLMM              $1.16M          $8.04M          $54.40M        
Raydium AMM               $873.30K        $4.81M          $25.59M        
PumpSwap                  $862.58K        $7.19M          $29.94M        
Meteora DAMM V2           $725.13K        $4.59M          $31.42M        
Jito MEV Tips             $698.36K        $6.16M          $34.00M        
Phantom Wallet            $528.13K        $3.68M          $16.79M        
💾 Saved raw: 

#### Get all Solana tokens list via Jupiter API

In [8]:
# Fetch Jupiter token list
url = "https://token.jup.ag/all"
resp = requests.get(url)
tokens = resp.json()

# Convert to DataFrame
df = pd.DataFrame(tokens)

# Select useful columns
jupiter_df = df[["address", "symbol", "name", "decimals", "logoURI"]]

#### Step 4: Collect CoinGecko Price and Supply Data

In [9]:

print("\n🔍 Collecting CoinGecko Price and Supply Data for Solana DeFi Protocols...")

def load_coingecko_list(cache_file='coingecko_coin_list.joblib'):
    """
    Fetch or load cached CoinGecko coin list
    """
    # Use the new cache loading method
    cached_data = load_cache(cache_file)
    if cached_data:
        return cached_data
    
    try:
        response = requests.get('https://api.coingecko.com/api/v3/coins/list')
        if response.status_code == 200:
            coin_list = response.json()
            save_cache(coin_list, cache_file, "CoinGecko coin list")
            return coin_list
        else:
            return []
    except Exception:
        return []

def get_coingecko_id_from_name(protocol_name, protocol_symbol, coin_list, chains, coin_cache, search_cache):
    """
    Try to find CoinGecko ID by first matching protocol symbol (exact match only, validated), then falling back to protocol name
    """
    if not API_KEYS.get('coingecko'):
        return None
        
    headers = {'X-Cg-Pro-Api-Key': API_KEYS['coingecko']}
    search_url = f"{API_ENDPOINTS['coingecko']['base_url']}/search"
    
    # Clean protocol name for fallback matching
    clean_name = protocol_name.lower().strip()
    
    # Step 1: Try exact symbol match (case-insensitive, validated)
    if protocol_symbol and isinstance(protocol_symbol, str):
        clean_symbol = protocol_symbol.lower().strip()
        # Validate symbol: alphabetic, 2-10 characters
        if re.match(r'^[a-zA-Z]{2,10}$', clean_symbol):
            for coin in coin_list:
                coin_symbol = coin.get('symbol', '').lower()
                coin_id = coin.get('id')
                
                if clean_symbol == coin_symbol:
                    # Check cache for coin details
                    if coin_id in coin_cache:
                        platforms = coin_cache[coin_id].get('platforms', {})
                    else:
                        coin_url = f"{API_ENDPOINTS['coingecko']['base_url']}/coins/{coin_id}"
                        try:
                            coin_data = make_request(coin_url, headers=headers, params={'localization': 'false'})
                            if coin_data:
                                coin_cache[coin_id] = coin_data
                                platforms = coin_data.get('platforms', {})
                            else:
                                platforms = {}
                        except Exception:
                            platforms = {}
                    
                    # Strict Solana check: must have 'solana' in platforms
                    if any('solana' in k.lower() for k in platforms.keys()):
                        return coin_id
    
    # Step 2: Fallback to name-based matching
    best_match = None
    best_score = 0.9  # Stricter threshold
    
    for coin in coin_list:
        coin_name = coin.get('name', '').lower()
        coin_symbol = coin.get('symbol', '').lower()
        coin_id = coin.get('id')
        
        # Skip known blockchain names
        if coin_id in ['aptos', 'solana', 'ethereum', 'binancecoin']:
            continue
        
        name_similarity = ratio(clean_name, coin_name) if len(clean_name) >= 4 and len(coin_name) >= 4 else 0
        symbol_similarity = ratio(clean_name, coin_symbol) if len(clean_name) >= 3 and len(coin_symbol) >= 3 else 0
        
        if (clean_name == coin_name or
            clean_name == coin_symbol or
            (clean_name in coin_name and len(clean_name) >= 4) or
            name_similarity > best_score or
            symbol_similarity > best_score):
            score = max(name_similarity, symbol_similarity, 1.0 if clean_name in coin_name else 0.0)
            if score > best_score:
                best_match = coin_id
                best_score = score
    
    if best_match:
        # Check cache for coin details
        if best_match in coin_cache:
            platforms = coin_cache[best_match].get('platforms', {})
        else:
            coin_url = f"{API_ENDPOINTS['coingecko']['base_url']}/coins/{best_match}"
            try:
                coin_data = make_request(coin_url, headers=headers, params={'localization': 'false'})
                if coin_data:
                    coin_cache[best_match] = coin_data
                    platforms = coin_data.get('platforms', {})
                else:
                    platforms = {}
            except Exception:
                platforms = {}
        
        # Strict Solana check
        if any('solana' in k.lower() for k in platforms.keys()):
            return best_match
    
    # Step 3: Fallback to API search (cached)
    if clean_name in search_cache:
        return search_cache[clean_name]
    
    params = {'query': clean_name}
    try:
        search_data = make_request(search_url, headers=headers, params=params)
        if search_data and 'coins' in search_data:
            best_match = None
            best_score = 0.9
            
            for coin in search_data['coins'][:3]:
                coin_name = coin.get('name', '').lower()
                coin_symbol = coin.get('symbol', '').lower()
                coin_id = coin.get('id')
                
                # Skip known blockchain names
                if coin_id in ['aptos', 'solana', 'ethereum', 'binancecoin']:
                    continue
                
                name_similarity = ratio(clean_name, coin_name) if len(clean_name) >= 4 and len(coin_name) >= 4 else 0
                symbol_similarity = ratio(clean_name, coin_symbol) if len(clean_name) >= 3 and len(coin_symbol) >= 3 else 0
                
                if (clean_name == coin_name or
                    clean_name == coin_symbol or
                    (clean_name in coin_name and len(clean_name) >= 4) or
                    name_similarity > best_score or
                    symbol_similarity > best_score):
                    score = max(name_similarity, symbol_similarity, 1.0 if clean_name in coin_name else 0.0)
                    if score > best_score:
                        best_match = coin_id
                        best_score = score
            
            if best_match:
                # Check Solana for search fallback
                if best_match in coin_cache:
                    platforms = coin_cache[best_match].get('platforms', {})
                else:
                    coin_url = f"{API_ENDPOINTS['coingecko']['base_url']}/coins/{best_match}"
                    try:
                        coin_data = make_request(coin_url, headers=headers, params={'localization': 'false'})
                        if coin_data:
                            coin_cache[best_match] = coin_data
                            platforms = coin_data.get('platforms', {})
                        else:
                            platforms = {}
                    except Exception:
                        platforms = {}
                
                if any('solana' in k.lower() for k in platforms.keys()):
                    search_cache[clean_name] = best_match
                    return best_match
    except Exception:
        pass
        
    return None

def batch_fetch_market_data(coingecko_ids, headers):
    """
    Fetch market data for multiple CoinGecko IDs in a single request
    """
    if not coingecko_ids:
        return {}
    
    market_data = {}
    batch_size = 100  # Safe for free-tier
    for i in range(0, len(coingecko_ids), batch_size):
        batch_ids = coingecko_ids[i:i + batch_size]
        url = f"{API_ENDPOINTS['coingecko']['base_url']}/coins/markets"
        params = {
            'vs_currency': 'usd',
            'ids': ','.join(batch_ids),
            'order': 'market_cap_desc',
            'per_page': batch_size,
            'page': 1,
            'sparkline': 'false',
            'price_change_percentage': '24h,7d,30d'
        }
        try:
            response = make_request(url, headers=headers, params=params)
            if response:
                for coin in response:
                    market_data[coin['id']] = {
                        'symbol': coin.get('symbol', '').upper(),
                        'current_price_usd': coin.get('current_price', 0) or 0,
                        'market_cap_usd': coin.get('market_cap', 0) or 0,
                        'total_volume_24h_usd': coin.get('total_volume', 0) or 0,
                        'price_change_24h_percent': coin.get('price_change_percentage_24h', 0) or 0,
                        'price_change_7d_percent': coin.get('price_change_percentage_7d', 0) or 0,
                        'price_change_30d_percent': coin.get('price_change_percentage_30d', 0) or 0,
                        'circulating_supply': coin.get('circulating_supply', 0) or 0,
                        'total_supply': coin.get('total_supply', 0) or 0,
                        'max_supply': coin.get('max_supply'),
                        'ath_usd': coin.get('ath', 0) or 0,
                        'atl_usd': coin.get('atl', 0) or 0,
                        'market_cap_rank': coin.get('market_cap_rank'),
                        'fully_diluted_valuation': coin.get('fully_diluted_valuation', 0) or 0
                    }
            time.sleep(0.2)  # Adjust to 0.5s for free-tier
        except Exception:
            pass
    return market_data

def collect_coingecko_data_for_solana_protocols():
    """
    Collect CoinGecko price data for Solana DeFi protocols found in TVL data
    """
    if tvl_df.empty:
        print("❌ No TVL data available. Please run the DefiLlama TVL collection first.")
        return {}
    
    if not API_KEYS.get('coingecko'):
        print("⚠️ CoinGecko API key not found in .env")
        return {}
    
    headers = {'X-Cg-Pro-Api-Key': API_KEYS['coingecko']}
    # Convert DataFrame rows to list of dictionaries
    solana_protocols = tvl_df.to_dict('records')
    
    coingecko_data = {}
    processed_count = 0
    successful_count = 0
    
    # Load caches using new cache methods
    coin_cache = load_cache('coingecko_coin_cache.joblib') or {}
    search_cache = load_cache('coingecko_search_cache.joblib') or {}
    
    # Load the CoinGecko coin list once
    coin_list = load_coingecko_list()
    print(f"📋 Loaded {len(coin_list)} coins from CoinGecko list")
    
    # Focus on top protocols by TVL
    top_protocols = sorted(solana_protocols, key=lambda x: x.get('tvl', 0), reverse=True)[:200]
    
    print(f"🎯 Processing top {len(top_protocols)} Solana protocols by TVL...")
    
    # Step 1: Collect CoinGecko IDs
    coingecko_ids = []
    protocol_map = {}
    
    for protocol in tqdm(top_protocols, desc="Matching protocols"):
        protocol_name = protocol.get('name', '').lower().strip()
        protocol_symbol = protocol.get('symbol', '')
        protocol_chains = protocol.get('chains', [])
        protocol_slug = protocol.get('slug', '')
        
        if not protocol_name:
            continue
            
        processed_count += 1
        
        # Find CoinGecko ID
        coingecko_id = get_coingecko_id_from_name(protocol.get('name'), protocol_symbol, coin_list, protocol_chains, coin_cache, search_cache)
        
        if coingecko_id:
            coingecko_ids.append(coingecko_id)
            protocol_map[coingecko_id] = {
                'protocol_name': protocol.get('name'),
                'protocol_slug': protocol_slug,
                'tvl': protocol.get('tvl', 0),
                'category': protocol.get('category', '')
            }
    
    # Step 2: Batch fetch market data
    print("📡 Fetching market data in batch...")
    market_data = batch_fetch_market_data(list(set(coingecko_ids)), headers)
    
    # Step 3: Populate coingecko_data
    for coingecko_id in market_data:
        if coingecko_id in protocol_map:
            protocol_info = protocol_map[coingecko_id]
            key = protocol_info['protocol_slug'] or protocol_info['protocol_name'].lower().replace(' ', '_')
            coingecko_data[key] = {
                'protocol_name': protocol_info['protocol_name'],
                'coingecko_id': coingecko_id,
                'symbol': market_data[coingecko_id]['symbol'],
                'current_price_usd': market_data[coingecko_id]['current_price_usd'],
                'market_cap_usd': market_data[coingecko_id]['market_cap_usd'],
                'total_volume_24h_usd': market_data[coingecko_id]['total_volume_24h_usd'],
                'price_change_24h_percent': market_data[coingecko_id]['price_change_24h_percent'],
                'price_change_7d_percent': market_data[coingecko_id]['price_change_7d_percent'],
                'price_change_30d_percent': market_data[coingecko_id]['price_change_30d_percent'],
                'circulating_supply': market_data[coingecko_id]['circulating_supply'],
                'total_supply': market_data[coingecko_id]['total_supply'],
                'max_supply': market_data[coingecko_id]['max_supply'],
                'ath_usd': market_data[coingecko_id]['ath_usd'],
                'atl_usd': market_data[coingecko_id]['atl_usd'],
                'market_cap_rank': market_data[coingecko_id]['market_cap_rank'],
                'fully_diluted_valuation': market_data[coingecko_id]['fully_diluted_valuation'],
                'tvl': protocol_info['tvl'],
                'category': protocol_info['category'],
                'collection_timestamp': datetime.now()
            }
            successful_count += 1
    
    # Save caches using new cache method
    save_cache(coin_cache, 'coingecko_coin_cache.joblib', "CoinGecko coin details cache")
    save_cache(search_cache, 'coingecko_search_cache.joblib', "CoinGecko search results cache")
    
    return coingecko_data, processed_count, successful_count

# Execute the collection
if 'tvl_df' in locals() and not tvl_df.empty:
    coingecko_data, processed_count, successful_count = collect_coingecko_data_for_solana_protocols()
    
    if coingecko_data:
        print(f"\n✅ Successfully collected CoinGecko data!")
        print(f"📊 Processed: {processed_count} protocols")
        print(f"🎯 Successful matches: {successful_count}")
        print(f"📈 Success rate: {(successful_count/processed_count*100):.1f}%")
        
        # Save raw data
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        save_raw_data(coingecko_data, f'solana_coingecko_enhanced_{timestamp}.joblib', 
                     'Enhanced CoinGecko data for Solana DeFi protocols')
        
        # Display sample of collected data
        print(f"\n📋 Sample of collected data:")
        print(f"{'Protocol':<25} {'Symbol':<8} {'Price':<12} {'Market Cap':<15} {'TVL':<15}")
        print("=" * 75)
        
        # Sort by market cap for display
        sorted_data = sorted(coingecko_data.items(), 
                           key=lambda x: x[1].get('market_cap_usd', 0), 
                           reverse=True)
        
        for protocol_key, data in sorted_data[:15]:  # Show top 15
            protocol_name = data.get('protocol_name', protocol_key)[:24]
            symbol = data.get('symbol', 'N/A')[:7]
            price = f"${data.get('current_price_usd', 0):.4f}"
            mcap = format_currency(data.get('market_cap_usd', 0))
            tvl = format_currency(data.get('tvl', 0))
            print(f"{protocol_name:<25} {symbol:<8} {price:<12} {mcap:<15} {tvl:<15}")
            
        # Create summary stats
        total_market_cap = sum(data.get('market_cap_usd', 0) for data in coingecko_data.values())
        total_tvl = sum(data.get('tvl', 0) for data in coingecko_data.values())
        
        print(f"\n📊 Portfolio Summary:")
        print(f"  • Total Market Cap: {format_currency(total_market_cap)}")
        print(f"  • Total TVL: {format_currency(total_tvl)}")
        print(f"  • Tokens with positive 24h change: {sum(1 for data in coingecko_data.values() if data.get('price_change_24h_percent', 0) > 0)}/{len(coingecko_data)}")
        
else:
    print("❌ TVL data not available. Please run the DefiLlama TVL collection cell first.")


🔍 Collecting CoinGecko Price and Supply Data for Solana DeFi Protocols...
📋 Loaded 18518 coins from CoinGecko list
🎯 Processing top 200 Solana protocols by TVL...


Matching protocols: 100%|██████████| 200/200 [02:17<00:00,  1.46it/s]


📡 Fetching market data in batch...
💾 Saved cache: CoinGecko coin details cache → ..\data\temp\coingecko_coin_cache.joblib
💾 Saved cache: CoinGecko search results cache → ..\data\temp\coingecko_search_cache.joblib

✅ Successfully collected CoinGecko data!
📊 Processed: 200 protocols
🎯 Successful matches: 80
📈 Success rate: 40.0%
💾 Saved raw: Enhanced CoinGecko data for Solana DeFi protocols → ..\data\api_responses\solana_coingecko_enhanced_20250902_100638.joblib

📋 Sample of collected data:
Protocol                  Symbol   Price        Market Cap      TVL            
BlackRock BUIDL           BUIDL    $1.0000      $2.40B          $2.41B         
Penguin                   PENGU    $0.0294      $1.85B          $30.32K        
Jupiter Lend              JUP      $0.4912      $1.53B          $490.09M       
Saros DLMM                SAROS    $0.3650      $958.11M        $1.75M         
Raydium AMM               RAY      $3.3900      $909.14M        $2.31B         
PancakeSwap AMM V3        

#### Step 5: Collect Helius Token Holder Data

In [10]:
print("\n🔍 Collecting Helius Token Holder Data for Solana DeFi Protocols...")
print("=" * 50)

# Helius API setup
helius_url = f"https://mainnet.helius-rpc.com/?api-key={API_KEYS['helius']}"
headers = {"Content-Type": "application/json"}

def get_solana_token_holders(jupiter_df, coingecko_data, tvl_df):
    """
    Collect top 10 largest accounts for each Solana protocol token matched in coingecko_data
    Uses Jupiter token list for contract addresses and tvl_df for symbol fallback
    Returns pandas DataFrame ready for charts and analysis
    """
    if coingecko_data is None or not coingecko_data:
        print("❌ No CoinGecko data available. Please run collect_coingecko_data.py first.")
        return pd.DataFrame()
    
    if jupiter_df.empty:
        print("❌ No Jupiter token data available. Please run Jupiter token list collection first.")
        return pd.DataFrame()
    
    if 'address' not in jupiter_df.columns:
        print("❌ Jupiter DataFrame missing 'address' column. Available columns:", jupiter_df.columns.tolist())
        return pd.DataFrame()
    
    if tvl_df.empty or 'symbol' not in tvl_df.columns:
        print("❌ tvl_df is empty or missing 'symbol' column. Available columns:", tvl_df.columns.tolist())
        return pd.DataFrame()
    
    # Create token mapping from coingecko_data and jupiter_df
    token_map = {}
    unmatched_tokens = []
    
    for protocol_key, data in coingecko_data.items():
        symbol = data.get('symbol', '').lower().strip()
        protocol_name = data.get('protocol_name', protocol_key)
        coingecko_id = data.get('coingecko_id', '')
        
        # Find matching token in Jupiter list
        jupiter_match = jupiter_df[jupiter_df['symbol'].str.lower() == symbol]
        
        if not jupiter_match.empty:
            # Take first match (assume most relevant)
            token_address = jupiter_match.iloc[0]['address']
            token_map[protocol_name] = token_address
        else:
            unmatched_tokens.append((protocol_name, symbol, coingecko_id))
    
    if unmatched_tokens:
        print(f"⚠️ {len(unmatched_tokens)} tokens not found in Jupiter list: {', '.join([f'{name} ({symbol})' for name, symbol, _ in unmatched_tokens[:5]])}")
        # Save unmatched tokens log to cache directory
        save_cache(unmatched_tokens, f'unmatched_tokens_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib', 
                  "Unmatched tokens log")
    
    all_holders_list = []
    successful_requests = 0
    failed_requests = 0
    token_logs = []  # Store per-token logs for debugging
    
    for token_name, token_address in tqdm(token_map.items(), desc="Fetching token holders"):
        try:
            # Prepare API payload
            payload = {
                "jsonrpc": "2.0",
                "id": "1",
                "method": "getTokenLargestAccounts",
                "params": [token_address]
            }
            
            # Make API request
            response = requests.post(helius_url, json=payload, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            if 'result' not in result or 'value' not in result['result']:
                token_logs.append(f"❌ Error fetching data for {token_name}: {result.get('error', 'No data returned')}")
                failed_requests += 1
                continue
            
            accounts = result['result']['value'][:10]  # Top 10 accounts
            
            if not accounts:
                token_logs.append(f"⚠️ No accounts found for {token_name}")
                failed_requests += 1
                continue
            
            # Get token symbol: prioritize coingecko_data['symbol'], then tvl_df['symbol'], then token_name
            coingecko_symbol = coingecko_data.get(token_name.lower().replace(' ', '_'), {}).get('symbol', '')
            tvl_symbol = ''
            if not coingecko_symbol:
                # Look up symbol in tvl_df
                tvl_match = tvl_df[tvl_df['name'].str.lower() == token_name.lower()]
                if not tvl_match.empty:
                    tvl_symbol = tvl_match.iloc[0]['symbol']
            
            token_symbol = coingecko_symbol or tvl_symbol or token_name
            
            # Store in list format for DataFrame
            for rank, account in enumerate(accounts, 1):
                all_holders_list.append({
                    'token_name': token_name,
                    'token_symbol': token_symbol,
                    'token_address': token_address,
                    'rank': rank,
                    'account_address': account['address'],
                    'ui_amount': account.get('uiAmount', 0),
                    'raw_amount': account.get('amount', '0'),
                    'decimals': account.get('decimals', 0),
                    'timestamp': datetime.now()
                })
            
            successful_requests += 1
            top_holder_amount = accounts[0].get('uiAmount', 0) if accounts else 0
            token_logs.append(f"✅ {token_name}: {len(accounts)} accounts, top holder: {top_holder_amount:,.0f} tokens")
            
            # Rate limiting for Helius API
            time.sleep(1.0)  # 1s delay to avoid rate limiting
            
        except Exception as e:
            token_logs.append(f"❌ Error processing {token_name}: {e}")
            failed_requests += 1
            continue
    
    # Save token logs to cache directory
    save_cache(token_logs, f'token_holder_logs_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib', 
              "Token holder collection logs")
    
    # Create pandas DataFrame
    df = pd.DataFrame(all_holders_list)
    
    if not df.empty:
        # Calculate percentage of top 10 for each token
        for token_name in df['token_name'].unique():
            token_mask = df['token_name'] == token_name
            token_total = df[token_mask]['ui_amount'].sum()
            
            if token_total > 0:
                df.loc[token_mask, 'percentage_of_top10'] = (df.loc[token_mask, 'ui_amount'] / token_total * 100)
            else:
                df.loc[token_mask, 'percentage_of_top10'] = 0
    
    print(f"\n✅ Successfully fetched data for {successful_requests} tokens, {failed_requests} failures")
    print(f"📊 DataFrame created with {len(df)} records")
    
    # Save as joblib
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    save_raw_data(df, f'solana_token_holders_{timestamp}.joblib', 'Solana token holders DataFrame')
    
    return df

# Execute the collection
holders_data = get_solana_token_holders(jupiter_df, coingecko_data, tvl_df)

if not holders_data.empty:
    print("Data saved to joblib file for further analysis")
else:
    print("\n❌ Failed to collect token holders data")

print("\n" + "=" * 50)


🔍 Collecting Helius Token Holder Data for Solana DeFi Protocols...
⚠️ 11 tokens not found in Jupiter list: Apollo Diversified Credit Securitize Fund (acred), VanEck Treasury Fund (vbill), DFDV Staked SOL (dfdvsol), Adrena Protocol (adx), FlashTrade (faf)
💾 Saved cache: Unmatched tokens log → ..\data\temp\unmatched_tokens_20250902_100657.joblib


Fetching token holders: 100%|██████████| 69/69 [02:42<00:00,  2.35s/it]

💾 Saved cache: Token holder collection logs → ..\data\temp\token_holder_logs_20250902_100939.joblib

✅ Successfully fetched data for 68 tokens, 1 failures
📊 DataFrame created with 626 records
💾 Saved raw: Solana token holders DataFrame → ..\data\api_responses\solana_token_holders_20250902_100940.joblib
Data saved to joblib file for further analysis

